<a href="https://colab.research.google.com/github/SmritiReddyy/MLOps_OCR_Only/blob/main/Latest_ENG_SPA_Translation6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# NEW INSTALLATION CELL

%%time
# ============================================================
# CLEAN + STABLE INSTALL CELL
# ============================================================

# First remove conflicting packages (clean state)
!pip uninstall -y numpy paddlepaddle paddleocr torch transformers sentencepiece 2>/dev/null

# Install stable NumPy FIRST (critical)
!pip install numpy==1.26.4

# --- Core Tools ---
!pip install -q pymupdf Pillow

# --- OCR Tools ---
!pip install -q pytesseract
!apt-get install -qq tesseract-ocr tesseract-ocr-spa tesseract-ocr-por > /dev/null 2>&1

# --- PaddleOCR ---
!pip install paddlepaddle==2.6.2
!pip install paddleocr==2.8.1

# --- Transformers / NLLB ---
!pip install torch==2.2.2
!pip install transformers==4.38.2 sentencepiece

# --- LangChain ---
!pip install langchain==0.0.352
!pip install -q langchain-community

print("\n" + "="*60)
print("✅ All tools installed successfully!")
print("="*60)


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: paddlepaddle 2.6.2
Uninstalling paddlepaddle-2.6.2:
  Successfully uninstalled paddlepaddle-2.6.2
Found existing installation: paddleocr 2.8.1
Uninstalling paddleocr-2.8.1:
  Successfully uninstalled paddleocr-2.8.1
Found existing installation: torch 2.2.2
Uninstalling torch-2.2.2:
  Successfully uninstalled torch-2.2.2
Found existing installation: sentencepiece 0.2.1
Uninstalling sentencepiece-0.2.1:
  Successfully uninstalled sentencepiece-0.2.1
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.25.0+cu128 requires to

  Using cached paddlepaddle-2.6.2-cp312-cp312-manylinux1_x86_64.whl.metadata (8.6 kB)
Using cached paddlepaddle-2.6.2-cp312-cp312-manylinux1_x86_64.whl (125.9 MB)
  Using cached paddleocr-2.8.1-py3-none-any.whl.metadata (19 kB)
Using cached paddleocr-2.8.1-py3-none-any.whl (407 kB)
  Using cached torch-2.2.2-cp312-cp312-manylinux1_x86_64.whl.metadata (25 kB)
Using cached torch-2.2.2-cp312-cp312-manylinux1_x86_64.whl (755.5 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.18.1 requires transformers, which is not installed.
sentence-transformers 5.2.3 requires transformers<6.0.0,>=4.41.0, which is not installed.
torchtune 0.6.1 requires sentencepiece, which is not installed.
torchaudio 2.10.0+cu128 requires torch==2.10.0, but you have torch 2.2.2 which is incompatible.
torchvision 0.25.0+cu128 requires torch==2.10.0, but you have torch 2.2.2 which is in


✅ All tools installed successfully!
CPU times: user 13.4 s, sys: 1.96 s, total: 15.4 s
Wall time: 1min 32s


In [2]:
import os
from pathlib import Path

WORK_DIR = Path("/content/courtaccess_benchmark")
WORK_DIR.mkdir(exist_ok=True)

# ============================================================
# OPTION A: Upload your own PDF
# ============================================================
from google.colab import files
print("Upload a government form PDF (or skip to use the generated sample):")
print("(Click 'Choose Files' below, or just run the next cell to use the sample)\n")

try:
    uploaded = files.upload()
    if uploaded:
        for fname, content in uploaded.items():
            pdf_path = WORK_DIR / fname
            pdf_path.write_bytes(content)
            print(f"✅ Uploaded: {pdf_path}")
            INPUT_PDF = str(pdf_path)
except KeyboardInterrupt:
    print("Skipped upload. Will use generated sample.")
    INPUT_PDF = None

Upload a government form PDF (or skip to use the generated sample):
(Click 'Choose Files' below, or just run the next cell to use the sample)



Saving test3.pdf to test3 (2).pdf
✅ Uploaded: /content/courtaccess_benchmark/test3 (2).pdf


In [3]:
# ============================================================
# NLLB TRANSLATION SETUP
# ============================================================

import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

device = torch.device("cuda")
# MODEL_NAME = "facebook/nllb-200-distilled-600M"
MODEL_NAME = "facebook/nllb-200-distilled-1.3B"

print("⏳ Loading NLLB model...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(device)
print("✅ NLLB ready")

# SOURCE_LANG = "spa_Latn"
# TARGET_LANG = "eng_Latn"

SOURCE_LANG = "eng_Latn"
TARGET_LANG = "spa_Latn"

# ── Patterns that should NEVER be translated ──────────────────
SKIP_PATTERNS = [
    r'^[\d\s\.\-\/\(\)\\,]+$',          # pure numbers/dates/punctuation
    r'^\s*[xX]\s*$',                      # signature placeholder "X"
    r'^https?://',                         # URLs
    r'^\$[\d,\.]+$',                       # dollar amounts
    r'^[\d]{1,2}[\/\-][\d]{1,2}[\/\-][\d]{2,4}$',  # date formats
    r'^[\-\s]*$',
    r'^\[[\d\.\(\)a-zA-Z\;\s]+\]', # [14.1(a)] style citations
    r'^\d+\.\s*$',                  # standalone "1." "2." etc
]

def should_translate(text: str) -> bool:
    t = text.strip()
    if not t:
        return False
    # Too short to translate meaningfully
    if len(t) <= 3:
        return False
    # All-caps short label (e.g. "DATE", "NO.", "BBO NO.", "DIVISION:")
    # Allow longer all-caps like full sentences in headings
    # if t.isupper() and len(t) < 25:
    if t.isupper() and len(t) < 12:
        return False

    # Matches a skip pattern
    for pat in SKIP_PATTERNS:
        if re.match(pat, t):
            return False
    return True


def split_into_sentences(text: str) -> list[str]:
    """
    Split on sentence-ending punctuation so NLLB gets complete
    sentences rather than fragments, which reduces hallucination.
    """
    # Split on . ! ? followed by space or end-of-string
    parts = re.split(r'(?<=[.!?])\s+', text.strip())
    return [p for p in parts if p.strip()]


def _raw_batch_translate(texts: list[str]) -> list[str]:
    """Low-level call to NLLB. Input texts should all be non-empty."""
    tokenizer.src_lang = SOURCE_LANG
    encoded = tokenizer(
        texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to(device)

    # Limit output length relative to input to curb hallucination
    input_len = encoded["input_ids"].shape[1]
    max_out = min(512, input_len + 40)

    generated_tokens = model.generate(
        **encoded,
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(TARGET_LANG),
        max_length=max_out,
        num_beams=4,
        early_stopping=True,
    )
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)


def batch_translate(texts: list[str]) -> list[str]:
    """
    Main translation function — drop-in replacement for your original.

    Changes vs original:
      1. Skips untranslatable strings (labels, numbers, short all-caps)
      2. Splits long strings into sentences before translating
      3. Length-ratio guard: if output is suspiciously longer or shorter
         than input, falls back to the original text
    """
    results = []

    for text in texts:
        # ── Skip guard ────────────────────────────────────────
        if not should_translate(text):
            results.append(text)
            continue

        # ── Sentence splitting for long text ──────────────────
        sentences = split_into_sentences(text)

        if len(sentences) <= 1:
            # Translate as single string
            translated_list = _raw_batch_translate([text])
            translated = translated_list[0]
        else:
            # Translate sentence by sentence, then rejoin
            translated_parts = _raw_batch_translate(sentences)
            translated = " ".join(translated_parts)

        # ── Length-ratio hallucination guard ──────────────────
        ratio = len(translated) / max(len(text), 1)
        if ratio > 2.8 or ratio < 0.15:
            # Suspiciously long or short — fall back to original
            print(f"   ⚠️  Hallucination suspected (ratio={ratio:.1f}), keeping original: '{text[:60]}'")
            results.append(text)
        else:
            results.append(translated)

    return results



⏳ Loading NLLB model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:949: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:949: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possibl

✅ NLLB ready


In [4]:
# ============================================================
# TASK 8: GROQ / LLaMA 3.3 70B LEGAL VERIFICATION LAYER
# Add this as a NEW CELL after your NLLB setup cell
# ============================================================

!pip install -q groq

import re
import time
import json
from groq import Groq
from google.colab import userdata

GROQ_API_KEY = userdata.get('GROQ_API_KEY')
groq_client = Groq(api_key=GROQ_API_KEY)

# ── Rate limit tracking ────────────────────────────────────
_groq_request_times = []

def _wait_for_rate_limit(max_per_minute=25):
    now = time.time()
    global _groq_request_times
    _groq_request_times = [t for t in _groq_request_times if now - t < 60]
    if len(_groq_request_times) >= max_per_minute:
        wait = 60 - (now - _groq_request_times[0]) + 1
        print(f"   ⏳ Groq rate limit — waiting {wait:.1f}s...")
        time.sleep(wait)
    _groq_request_times.append(time.time())


def _strip_code_fences(text: str) -> str:
    """Remove markdown code fences without using backtick literals in regex."""
    fence = chr(96) * 3          # builds ``` from char code — no syntax error
    text = text.strip()
    if text.startswith(fence + "json"):
        text = text[7:]          # remove ```json
    elif text.startswith(fence):
        text = text[3:]          # remove ```
    if text.endswith(fence):
        text = text[:-3]         # remove closing ```
    return text.strip()


# ── Patterns that indicate NLLB is likely wrong ───────────────
HIGH_RISK_PATTERNS = [
    r'Mass\.\s+R\.',                 # Mass. R. Crim. P. citations
    r'\[[\d\.\(\)a-zA-Z\;\s]+\]',   # [14.1(a)] style citations
    r'\bCommonwealth\b',             # legal proper noun
    r'\bwaiver\b|\bwaive\b',        # legal terms
    r'\bdefendant\b|\bplaintiff\b', # legal parties
    r'\bdocket\b',                   # court term
    r'G\.L\.\s+c\.',                # G.L. c. 263 citations
    r'Paragraph\s+\d+',            # paragraph references
    r'§\s*\d+',                     # section symbols
    r'^\d+\.',              # numbered list items "1." "2." etc
    r'\breciprocal\b',      # catches the automatic reciprocal discovery line
    r'\bautomatic\b',
]

def is_high_risk(original: str) -> bool:
    """Returns True if this span is likely to be mistranslated by NLLB."""
    for pat in HIGH_RISK_PATTERNS:
        if re.search(pat, original, re.IGNORECASE):
            return True
    return False


def verify_legal_translation(original_en: str, translated_es: str) -> dict:
    """Single-span verification — used only for the connection test."""
    if len(original_en.strip()) < 20:
        return {"approved": True, "issues": [], "corrected": translated_es}

    _wait_for_rate_limit()

    prompt = (
        "You are a legal translation verifier for Massachusetts court documents.\n"
        "Check if this Spanish translation of English legal text is accurate.\n\n"
        f"ENGLISH: {original_en}\n"
        f"SPANISH: {translated_es}\n\n"
        "Respond in this exact JSON format with no extra text:\n"
        '{"approved": true or false, "issues": ["issue 1"], '
        '"corrected": "corrected Spanish or same as input if approved"}'
    )

    try:
        response = groq_client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1,
            max_tokens=512,
        )
        raw = _strip_code_fences(response.choices[0].message.content)
        return json.loads(raw)
    except Exception as e:
        print(f"   ⚠️ Groq error: {e}")
        return {"approved": True, "issues": [], "corrected": translated_es}


def verify_page_translations(original_spans: list,
                              translated_spans: list,
                              batch_size: int = 8) -> list:
    """
    Efficient long-term verification:
    - Only sends HIGH-RISK spans to Groq (citations, legal terms)
    - Batches them into a single call returning a JSON array
    - Low-risk spans pass through with zero API calls
    For a 400-span document this uses ~3-4 Groq calls total.
    """
    verified = list(translated_spans)

    high_risk_indices = [
        i for i, orig in enumerate(original_spans)
        if should_translate(orig) and is_high_risk(orig)
    ]

    if not high_risk_indices:
        print(f"   ✅ No high-risk spans found, skipping Groq")
        return verified

    print(f"   🔍 {len(high_risk_indices)}/{len(original_spans)} spans flagged as high-risk")

    for batch_start in range(0, len(high_risk_indices), batch_size):
        batch_indices = high_risk_indices[batch_start:batch_start + batch_size]
        orig_batch = [original_spans[i] for i in batch_indices]
        trans_batch = [translated_spans[i] for i in batch_indices]

        _wait_for_rate_limit()

        pairs = "\n".join(
            f"{k+1}. EN: {o}\n   ES: {t}"
            for k, (o, t) in enumerate(zip(orig_batch, trans_batch))
        )

        prompt = (
            "You are a legal translator verifying English to Spanish translations "
            "of Massachusetts court documents.\n"
            "For each pair, check if the Spanish is legally accurate.\n"
            "Pay special attention to: legal citations (Mass. R. Crim. P.), "
            "legal terms (Commonwealth, defendant, waiver, docket), "
            "section references (G.L. c. X, § Y), and paragraph numbers.\n\n"
            "Return ONLY a JSON array of corrected Spanish strings, one per pair, "
            "in the same order. If a translation is already correct, return it unchanged.\n"
            "No explanation, no markdown, just the JSON array.\n\n"
            f"PAIRS:\n{pairs}\n\n"
            'Return format: ["corrected span 1", "corrected span 2", ...]'
        )

        try:
            response = groq_client.chat.completions.create(
                model="llama-3.3-70b-versatile",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.1,
                max_tokens=1024,
            )
            raw = _strip_code_fences(response.choices[0].message.content)
            corrected_list = json.loads(raw)

            if isinstance(corrected_list, list) and len(corrected_list) == len(batch_indices):
                for idx, corrected in zip(batch_indices, corrected_list):
                    if corrected and corrected.strip():
                        ratio = len(corrected) / max(len(translated_spans[idx]), 1)
                        if 0.2 <= ratio <= 4.0:
                            verified[idx] = corrected.strip()
                print(f"   ✅ Batch {batch_start//batch_size + 1}: corrected {len(batch_indices)} spans")
            else:
                print(f"   ⚠️  Batch {batch_start//batch_size + 1}: response length mismatch, keeping NLLB")

        except Exception as e:
            print(f"   ⚠️  Groq error on batch: {e} — keeping NLLB output")

    return verified


# ── Test connection ────────────────────────────────────────
print("🧪 Testing Groq connection...")
test = verify_legal_translation(
    "The defendant waives the right to jury trial.",
    "El acusado renuncia al derecho a juicio con jurado."
)
print(f"   approved={test['approved']}, issues={test['issues']}")
print("✅ Groq ready" if test['approved'] else f"⚠️ Groq flagged: {test['issues']}")

🧪 Testing Groq connection...
   approved=True, issues=[]
✅ Groq ready


In [5]:
# ============================================================
# TASK 9: MICROSOFT PRESIDIO PII REDACTION
# Add this as a NEW CELL after your Groq verification cell
# ============================================================

!pip install -q presidio-analyzer presidio-anonymizer
!python -m spacy download en_core_web_lg -q

from presidio_analyzer import AnalyzerEngine, RecognizerRegistry
from presidio_analyzer.nlp_engine import NlpEngineProvider
from presidio_anonymizer import AnonymizerEngine
from presidio_anonymizer.entities import OperatorConfig
import re

# ── Initialize Presidio ───────────────────────────────────
print("⏳ Loading Presidio...")
analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()
print("✅ Presidio ready")

# ── PII types to detect and redact ───────────────────────
# These are the entity types Presidio will look for.
# For legal documents we want to catch real PII but
# PRESERVE legal identifiers like case numbers and docket numbers.
PII_ENTITIES = [
    "PERSON",           # names like "Sunny Yadav"
    "PHONE_NUMBER",     # phone numbers
    "EMAIL_ADDRESS",    # emails
    "US_SSN",           # social security numbers
    "CREDIT_CARD",      # credit card numbers
    "US_PASSPORT",      # passport numbers
    "US_DRIVER_LICENSE",
    "IP_ADDRESS",
    "DATE_TIME",        # dates — optional, set REDACT_DATES=False to keep
]

# Set to False if you want to keep dates visible in output
REDACT_DATES = False
if not REDACT_DATES and "DATE_TIME" in PII_ENTITIES:
    PII_ENTITIES.remove("DATE_TIME")

# ── Patterns to NEVER redact (legal document identifiers) ─
PRESERVE_PATTERNS = [
    r'G\.L\.\s+c\.\s+\d+',          # G.L. c. 263
    r'§\s*\d+[A-Z]?',               # § 26A
    r'\[\d+[\.\(\)a-zA-Z\;]+\]',    # [14.1(a)]
    r'Mass\.\s+R\.\s+Crim\.',       # Mass. R. Crim. P.
    r'Paragraph\s+\d+',             # Paragraph 17
    r'TC\d+',                        # form numbers like TC0051
    r'P\.\s*\d+\.\d+',              # P. 14.2
]

def should_preserve(text: str) -> bool:
    """Returns True if this text contains legal identifiers that must not be redacted."""
    for pat in PRESERVE_PATTERNS:
        if re.search(pat, text, re.IGNORECASE):
            return True
    return False


def redact_pii(text: str, language: str = "en") -> tuple:
    """
    Detect and redact PII from a string.

    Args:
        text: input text (English or Spanish)
        language: "en" for English spans, "es" for Spanish
                  Note: Presidio works best on English.
                  For Spanish, we run English detection as a fallback
                  since most PII patterns (phones, SSNs) are language-agnostic.

    Returns:
        (redacted_text, list of entity types found)
    """
    if not text.strip() or should_preserve(text):
        return text, []

    # Presidio works best with English — for Spanish text,
    # use English analyzer since PII patterns are mostly language-agnostic
    try:
        results = analyzer.analyze(
            text=text,
            entities=PII_ENTITIES,
            language="en"   # always use English analyzer
        )
    except Exception:
        return text, []

    if not results:
        return text, []

    found_entities = list(set(r.entity_type for r in results))

    # Anonymize — replace PII with [ENTITY_TYPE] placeholder
    anonymized = anonymizer.anonymize(
        text=text,
        analyzer_results=results,
        operators={
            entity: OperatorConfig("replace", {"new_value": f"[{entity}]"})
            for entity in found_entities
        }
    )

    return anonymized.text, found_entities


def redact_spans(original_texts: list, translated_texts: list) -> list:
    """
    Run PII redaction on translated spans.
    Uses original English text to detect PII, then applies
    redaction at the same position in the translated text.

    This is more accurate than running Presidio on Spanish directly
    since Presidio's models are trained primarily on English.

    Returns:
        list of redacted translated texts
    """
    redacted = list(translated_texts)
    total_redacted = 0

    for i, (orig, trans) in enumerate(zip(original_texts, translated_texts)):
        # Detect PII in original English
        _, entities = redact_pii(orig, language="en")

        if not entities:
            continue

        # Apply redaction to translated Spanish text
        redacted_trans, _ = redact_pii(trans, language="en")
        redacted[i] = redacted_trans
        total_redacted += 1
        print(f"   🔒 Span {i}: redacted {entities} → '{trans[:40]}...' → '{redacted_trans[:40]}...'")

    if total_redacted == 0:
        print(f"   ✅ No PII found in {len(original_texts)} spans")
    else:
        print(f"   🔒 Redacted PII in {total_redacted}/{len(original_texts)} spans")

    return redacted


# ── Test Presidio on known PII ─────────────────────────────
print("\n🧪 Testing Presidio...")
test_cases = [
    "Sunny Yadav, Aditi Putrevu, Devasai Choppa",
    "Phone No: 8573519565",
    "75 Saint Alphonsus St, Apt 1215, Boston, MA 02120",
    "G.L. c. 263, § 6",        # should NOT be redacted
    "Mass. R. Crim. P. 14.2",  # should NOT be redacted
]
for t in test_cases:
    redacted, entities = redact_pii(t)
    status = "🔒" if entities else "✅"
    print(f"   {status} '{t[:50]}' → '{redacted[:50]}' {entities if entities else ''}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 3.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
⏳ Loading Presidio...


✅ Presidio ready

🧪 Testing Presidio...
   🔒 'Sunny Yadav, Aditi Putrevu, Devasai Choppa' → '[PERSON], [PERSON], [PERSON]' ['PERSON']
   🔒 'Phone No: 8573519565' → 'Phone No: [PHONE_NUMBER]' ['US_DRIVER_LICENSE', 'PHONE_NUMBER']
   🔒 '75 Saint Alphonsus St, Apt 1215, Boston, MA 02120' → '75 [PERSON], Apt 1215, Boston, MA 02120' ['PERSON']
   ✅ 'G.L. c. 263, § 6' → 'G.L. c. 263, § 6' 
   ✅ 'Mass. R. Crim. P. 14.2' → 'Mass. R. Crim. P. 14.2' 


In [6]:
# ============================================================
# UNIFIED RECONSTRUCTION — Handles Digital, Scanned & Hybrid PDFs
# ============================================================
import os
import sys
import pymupdf
from paddleocr import PaddleOCR

# Disable PaddleOCR's slow model source check
os.environ["PADDLE_PDX_DISABLE_MODEL_SOURCE_CHECK"] = "True"

# --- PaddleOCR Fix for Colab ---
# Attempt to reset PaddleX state if it was left initialized by a failed run
try:
    # We access the module directly from sys.modules if it exists to avoid triggering __init__
    if 'paddlex.repo_manager.core' in sys.modules:
        import paddlex.repo_manager.core as pdx_core
        # Try to reset internal flags if they exist (heuristics)
        if hasattr(pdx_core, '_GlobalContext'):
            pdx_core._GlobalContext._initialized = False
            print("🔄 Attempted to reset PaddleX state.")
except Exception:
    pass

PaddleOCR = None  # Will be set if import succeeds
try:
    from paddleocr import PaddleOCR
    print(f"✅ PaddleOCR: available")
except Exception as e:
    PaddleOCR = None
    print(f"⚠️ PaddleOCR unavailable: {e}")
    print(f"   Scanned pages will be skipped. RESTART RUNTIME to fix.")

OCR_DPI = 300
CONFIDENCE_THRESHOLD = 0.85

font_map = {
    "Helvetica": "helv", "Helvetica-Bold": "hebo",
    "Helvetica-Oblique": "heit", "Helvetica-BoldOblique": "hebi",
    "Times-Roman": "tiro", "Times-Bold": "tibo",
    "Times-Italic": "tiit", "Times-BoldItalic": "tibi",
    "Courier": "cour", "Courier-Bold": "cobo",
    "Courier-Oblique": "coit", "Courier-BoldOblique": "cobi",
    "Symbol": "symb", "ZapfDingbats": "zadb",
}

def get_font_code(span, font_map):
    fontname = span.get("font", "")
    flags = span.get("flags", 0)
    direct = font_map.get(fontname)
    if direct:
        return direct
    is_bold = bool(flags & 16) or "bold" in fontname.lower()
    is_italic = bool(flags & 2) or "italic" in fontname.lower() or "oblique" in fontname.lower()
    is_mono = bool(flags & 8) or "courier" in fontname.lower() or "mono" in fontname.lower()
    is_serif = bool(flags & 4) or "times" in fontname.lower() or "serif" in fontname.lower()
    if is_mono:
        if is_bold and is_italic: return "cobi"
        if is_bold: return "cobo"
        if is_italic: return "coit"
        return "cour"
    elif is_serif:
        if is_bold and is_italic: return "tibi"
        if is_bold: return "tibo"
        if is_italic: return "tiit"
        return "tiro"
    else:
        if is_bold and is_italic: return "hebi"
        if is_bold: return "hebo"
        if is_italic: return "heit"
        return "helv"

def estimate_fontsize(text, target_width, fontname="helv"):
    font = pymupdf.Font(fontname)
    best_size, best_diff = 5, float('inf')
    for s10 in range(50, 260, 2):
        size = s10 / 10.0
        w = font.text_length(text, fontsize=size)
        diff = abs(w - target_width)
        if diff < best_diff:
            best_diff = diff
            best_size = size
        if w > target_width * 1.1:
            break
    return best_size

def analyze_page(page):
    """Determine if a page is digital, scanned, or OCR'd scan."""
    td = page.get_text("dict", flags=pymupdf.TEXTFLAGS_TEXT)
    span_count = 0
    img_blocks = 0
    for b in td["blocks"]:
        if b["type"] == 1:
            img_blocks += 1
        elif b["type"] == 0:
            for line in b["lines"]:
                for span in line["spans"]:
                    if span["text"].strip():
                        span_count += 1

    images = page.get_images(full=True)
    drawings = page.get_drawings()

    # Check image coverage using multiple methods
    page_area = page.rect.width * page.rect.height
    total_img_area = 0

    # Method 1: get_image_rects (works for most PDFs)
    for img in images:
        xref = img[0]
        try:
            rects = page.get_image_rects(xref)
            for r in rects:
                total_img_area += r.width * r.height
        except:
            pass

    # Method 2: text dict image blocks (fallback)
    if total_img_area == 0:
        for b in td["blocks"]:
            if b["type"] == 1:
                bbox = b["bbox"]
                total_img_area += (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])

    # Method 3: check raw image dimensions (fallback for embedded scans)
    # If get_image_rects fails but images exist with large pixel dimensions,
    # estimate coverage from the image size vs page size
    has_large_image = False
    if total_img_area == 0 and len(images) > 0:
        for img in images:
            img_w, img_h = img[2], img[3]
            if img_w > 1000 and img_h > 1000:
                has_large_image = True
                # Estimate: a scan image this large likely fills the page
                total_img_area = page_area * 0.9
                break

    img_coverage = total_img_area / page_area if page_area > 0 else 0

    # Classification logic:
    #
    # DIGITAL: has text spans, has vector drawings (lines/borders/boxes), small images
    # SCANNED: few text spans, large image, no vector drawings
    # OCR'd SCAN: has text spans (OCR layer), large image OR no drawings, no vector art
    #
    # The KEY distinguisher: vector drawings.
    # Digital court forms always have dozens of drawn lines/borders.
    # Scanned pages have ZERO vector drawings — everything is pixels.

    is_scanned = False
    page_type = "DIGITAL"

    if span_count < 10 and len(images) > 0:
        # Classic scan: no text, has image
        is_scanned = True
        page_type = "SCANNED"
    elif img_coverage > 0.5 and len(drawings) < 5:
        # Large image covering page + almost no vector drawings = scan
        is_scanned = True
        page_type = "OCR'd SCAN" if span_count >= 10 else "SCANNED"
    elif len(drawings) < 5 and span_count > 10 and (has_large_image or len(images) > 0):
        # No vector drawings + has text + has images = OCR'd scan
        # Real digital PDFs with text ALWAYS have vector drawings for borders/lines
        is_scanned = True
        page_type = "OCR'd SCAN"

    return {
        "is_scanned": is_scanned,
        "page_type": page_type,
        "span_count": span_count,
        "images": len(images),
        "img_blocks": img_blocks,
        "drawings": len(drawings),
        "img_coverage": round(img_coverage * 100, 1),
        "has_large_image": has_large_image,
    }


def fit_fontsize(text, original_size, bbox_width, fontname="helv", min_size=4.0):
    """
    Returns a font size <= original_size such that text fits within bbox_width.
    Falls back to min_size if it still doesn't fit.
    """
    font = pymupdf.Font(fontname)
    size = original_size
    while size >= min_size:
        w = font.text_length(text, fontsize=size)
        if w <= bbox_width * 0.95:  # allow 5% overflow tolerance
            return size
        size -= 0.5
    return min_size


def fit_text_in_rect(page, rect, text, fontname="helv", color=(0, 0, 0),
                     original_size=10.0, min_size=4.0):
    """
    Insert text into a rect, scaling font down to fit width.
    If text still can't fit at min_size, truncate with ellipsis.
    """
    bbox_width = rect.width
    bbox_height = rect.height

    fitted_size = fit_fontsize(text, original_size, bbox_width, fontname, min_size)

    font = pymupdf.Font(fontname)
    # Truncate if even at min size it overflows
    if font.text_length(text, fontsize=fitted_size) > bbox_width * 1.05:
        # Truncate character by character
        for end in range(len(text), 0, -1):
            truncated = text[:end] + "…"
            if font.text_length(truncated, fontsize=fitted_size) <= bbox_width:
                text = truncated
                break

    # Vertical centering: baseline is ~0.78 of height for most fonts
    origin = pymupdf.Point(rect.x0, rect.y0 + min(rect.height * 0.78, rect.height - 1))

    try:
        page.insert_text(origin, text, fontsize=fitted_size, fontname=fontname, color=color)
    except Exception:
        pass

# ============================================================
# UPDATED reconstruct_digital_page — with Groq verification
# ============================================================

def reconstruct_digital_page(out_page, font_map, use_verification=True):
    text_dict = out_page.get_text("dict", flags=pymupdf.TEXTFLAGS_TEXT)
    all_spans = []

    for block in text_dict["blocks"]:
        if block["type"] != 0:
            continue
        for line in block["lines"]:
            for span in line["spans"]:
                if span["text"].strip():
                    all_spans.append(span)

    if not all_spans:
        return 0

    # Step 1: Collect and translate
    original_texts = [span["text"] for span in all_spans]
    translated_texts = batch_translate(original_texts)

    # Step 2: Groq verification (only on translatable spans)
    if use_verification:
        print(f"   🔍 Running Groq verification on {len(all_spans)} spans...")
        translatable_indices = [
            i for i, t in enumerate(original_texts) if should_translate(t)
        ]
        orig_to_verify = [original_texts[i] for i in translatable_indices]
        trans_to_verify = [translated_texts[i] for i in translatable_indices]
        if orig_to_verify:
            verified = verify_page_translations(orig_to_verify, trans_to_verify, batch_size=8)
            for idx, verified_text in zip(translatable_indices, verified):
                translated_texts[idx] = verified_text

    # Step 3: Presidio PII redaction
    print(f"   🔒 Running PII redaction...")
    translated_texts = redact_spans(original_texts, translated_texts)

    # Step 4: Redact originals + reinsert
    for span in all_spans:
        out_page.add_redact_annot(pymupdf.Rect(span["bbox"]), fill=(1, 1, 1))
    out_page.apply_redactions(images=pymupdf.PDF_REDACT_IMAGE_NONE)

    # Step 4: Reinsert verified translations
    count = 0
    for span, text in zip(all_spans, translated_texts):
        if not text.strip():
            continue

        bbox = pymupdf.Rect(span["bbox"])
        color = pymupdf.sRGB_to_rgb(span["color"])
        fnt = get_font_code(span, font_map)
        original_size = span["size"]

        fitted_size = fit_fontsize(text, original_size, bbox.width, fnt)
        origin = pymupdf.Point(span["origin"])
        if fitted_size < original_size * 0.6:
            origin = pymupdf.Point(bbox.x0, bbox.y0 + bbox.height * 0.78)

        try:
            out_page.insert_text(origin, text, fontsize=fitted_size,
                                  fontname=fnt, color=color)
            count += 1
        except Exception:
            try:
                out_page.insert_text(origin, text, fontsize=fitted_size,
                                      fontname="helv", color=color)
                count += 1
            except Exception:
                pass

    return count

# ============================================================
# UPDATED: reconstruct_scanned_page
# ============================================================

def reconstruct_scanned_page(out_page, ocr_engine, page_num_for_log):
    zoom = OCR_DPI / 72
    pix = out_page.get_pixmap(matrix=pymupdf.Matrix(zoom, zoom))
    img_bytes = pix.tobytes("png")

    import tempfile, os
    tmp = tempfile.NamedTemporaryFile(suffix=".png", delete=False)
    tmp.write(img_bytes)
    tmp.close()

    result = ocr_engine.ocr(tmp.name, cls=True)
    os.unlink(tmp.name)

    detections = result[0] if result and result[0] else []
    if not detections:
        return 0

    page_w, page_h = out_page.rect.width, out_page.rect.height
    scale_x = page_w / pix.width
    scale_y = page_h / pix.height

    # Filter by confidence and build regions
    regions = []
    for det in detections:
        bbox_px = det[0]
        text = det[1][0]
        conf = det[1][1]
        # if conf < CONFIDENCE_THRESHOLD:
        if conf < 0.80 or len(text.strip()) < 3:
            continue
        xs = [p[0] for p in bbox_px]
        ys = [p[1] for p in bbox_px]
        rect = pymupdf.Rect(
            min(xs) * scale_x - 1, min(ys) * scale_y - 1,
            max(xs) * scale_x + 1, max(ys) * scale_y + 1,
        )
        regions.append({"rect": rect, "text": text})

    if not regions:
        return 0

    # White-out all regions
    shape = out_page.new_shape()
    for r in regions:
        shape.draw_rect(r["rect"])
        shape.finish(color=None, fill=(1, 1, 1))
    shape.commit(overlay=True)

    # Batch translate
    original_texts = [r["text"] for r in regions]
    translated_texts = batch_translate(original_texts)


    # After translated_texts = batch_translate(original_texts)
    original_texts_list = [r["text"] for r in regions]
    translated_texts = redact_spans(original_texts_list, translated_texts)

    # Insert with size fitting
    count = 0
    for r, text in zip(regions, translated_texts):
        rect = r["rect"]
        if not text.strip():
            continue

        # Estimate font size from original box height, then fit to width
        initial_size = rect.height * 0.75
        fitted_size = fit_fontsize(text, initial_size, rect.width, "helv", min_size=4.0)
        fitted_size = max(4.0, min(fitted_size, rect.height * 0.9))

        origin = pymupdf.Point(rect.x0, rect.y0 + rect.height * 0.78)
        try:
            out_page.insert_text(origin, text, fontsize=fitted_size,
                                  fontname="helv", color=(0, 0, 0))
            count += 1
        except Exception:
            pass

    return count

    # Collect OCR texts
    original_texts = [r["text"] for r in regions]

    # Batch translate
    translated_texts = batch_translate(original_texts)

    # Insert translated text
    count = 0
    for r, text in zip(regions, translated_texts):
        rect = r["rect"]

        if not text.strip():
            continue

        fontsize = estimate_fontsize(text, rect.width)
        fontsize = max(5, min(fontsize, rect.height * 0.9))
        origin = pymupdf.Point(rect.x0, rect.y0 + rect.height * 0.78)

        try:
            out_page.insert_text(origin, text, fontsize=fontsize, fontname="helv", color=(0, 0, 0))
            count += 1
        except:
            pass



# ============================================================
# MAIN: Process all pages with auto-detection
# ============================================================
src_doc = pymupdf.open(INPUT_PDF)
out_doc = pymupdf.open()
out_doc.insert_pdf(src_doc)
src_doc.close()

# Lazy-init OCR only if needed
ocr_engine = None

digital_pages = 0
scanned_pages = 0

for pg in range(len(out_doc)):
    out_page = out_doc[pg]
    info = analyze_page(out_page)

    if not info["is_scanned"]:
        # DIGITAL PAGE
        print(f"\n📄 Page {pg+1}/{len(out_doc)}: {info['page_type']} ({info['span_count']} spans, {info['drawings']} drawings, {info['img_coverage']}% img coverage)")
        count = reconstruct_digital_page(out_page, font_map)
        print(f"   ✅ Redacted & re-inserted {count} spans")
        digital_pages += 1
    else:
        # SCANNED PAGE
        print(f"\n📷 Page {pg+1}/{len(out_doc)}: {info['page_type']} ({info['span_count']} spans, {info['images']} images → using OCR)")
        if ocr_engine is None:
            if PaddleOCR is None:
                print("   ❌ PaddleOCR not available — skipping scanned page. RESTART RUNTIME.")
                scanned_pages += 1
                continue
            print("  ⏳ Initializing PaddleOCR (first scanned page)...")
            # ocr_engine = PaddleOCR(use_angle_cls=True, lang='en')
            #Change 2
            ocr_engine = PaddleOCR(use_textline_orientation=True,lang='en')

        count = reconstruct_scanned_page(out_page, ocr_engine, pg)
        print(f"   ✅ OCR'd & inserted {count} text regions")
        scanned_pages += 1

UNIFIED_PDF = str(WORK_DIR / "unified_reconstructed.pdf")
out_doc.save(UNIFIED_PDF)
out_doc.close()

print(f"\n{'='*60}")
print(f"🎉 UNIFIED RECONSTRUCTION COMPLETE")
print(f"   Digital pages: {digital_pages}")
print(f"   Scanned pages: {scanned_pages}")
print(f"   Output: {UNIFIED_PDF}")
print(f"{'='*60}")

✅ PaddleOCR: available

📄 Page 1/2: DIGITAL (115 spans, 168 drawings, 0.0% img coverage)
   ⚠️  Hallucination suspected (ratio=5.8), keeping original: 'www.mass.gov/dph/rvrs'
   🔍 Running Groq verification on 115 spans...
   🔍 3/95 spans flagged as high-risk
   ✅ Batch 1: corrected 3 spans
   🔒 Running PII redaction...
   🔒 Span 42: redacted ['PERSON'] → 'Dirección postal ...' → 'Dirección postal ...'
   🔒 Span 104: redacted ['PERSON'] → 'Aidan Quinn. ¿ Qué quieres decir?...' → '[PERSON]. ¿ Qué quieres decir?...'
   🔒 Span 106: redacted ['PERSON'] → 'Carter Baizen. ¿ Qué quieres decir?...' → '[PERSON]. ¿ Qué quieres decir?...'
   🔒 Span 108: redacted ['PERSON'] → 'Carter Baizen. ¿ Qué quieres decir?...' → '[PERSON]. ¿ Qué quieres decir?...'
   🔒 Span 110: redacted ['PERSON'] → 'Es una desconocida....' → 'Es una desconocida....'
   🔒 Span 112: redacted ['PERSON'] → 'Es un desconocido....' → 'Es un desconocido....'
   🔒 Span 114: redacted ['PERSON'] → 'Es un desconocido....' → 'Es un des